In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from keras.models import load_model
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
IMG_SIZE = 100
NUM_FRAME= 10


In [7]:
cnn = load_model('CNN-BiLSTM.h5')
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 10, 100, 100, 64)  1792      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 98, 98, 64)    36928     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 49, 49, 64)    0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 47, 47, 64)    36928     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 10, 23, 23, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 21, 21, 64)    36928     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 10, 10, 64)   

In [11]:
def create_dataset():
    dataset = []
    image = []
    limit = 0
    c = 0

    for file in tqdm(os.listdir('./Peliculas/Dataframes/')):
        path = os.path.join('./Peliculas/Dataframes/', file)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image.append(np.array(img))
        limit += 1
        c += 1
        if c == NUM_FRAME:
            c = 0
            if limit < 4792:
                dataset.append([image, np.array([1, 0])])
            elif limit >= 4792:
                dataset.append([image, np.array([0, 1])])
            image = []
    
    shuffle(dataset)
    np.save('dataset_movie.npy', dataset)
    return dataset

In [12]:
data = create_dataset()

100%|██████████| 9841/9841 [00:39<00:00, 249.61it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
data = np.load('dataset_movie.npy', allow_pickle=True)

In [23]:
train, test = train_test_split(data, train_size= 0.9, shuffle=True)

In [24]:
train[5][1]

array([0, 1])

In [4]:
X = np.array([i[0] for i in data]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in data])

#x_valid = np.array([i[0] for i in test]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
#y_valid = np.array([i[1] for i in test])

In [5]:
X = X.astype('float32')/255
#x_valid = x_valid.astype('float32')/255

In [27]:
print(Y.shape)
print(y_valid.shape)
print(X.shape)
print(x_valid.shape)

print(y_valid[7])

(885, 2)
(99, 2)
(885, 10, 100, 100, 3)
(99, 10, 100, 100, 3)
[0 1]


In [8]:
cnn.evaluate(x = X, y =Y, batch_size= 5, verbose = 1)

197/197 [==============================] - 10s 42ms/step - loss: 1.9003 - accuracy: 0.4573


[1.9002829790115356, 0.4573170840740204]